- Diana Zaray Corado #191025
- Pablo Alejandro Méndez #19195
- Orlando Osberto Cabrera #19943
- Javier Alejandro Mejía Alecio #20304
- Erick Raúl Alvarez Melgar #20900

# Proyecto - Análisis Exploratorio

Guatemala es conocido como el país de la eterna primavera, debido a su diversidad de flora y fauna, además de contar
con un clima conocido como templado, es decir no existen climas extremos a lo largo del año. Sin embargo, así como
es conocido por su belleza natural, también es fuertemente reconocible por la corrupción y los bajos índices de calidad
de vida, y uno de ellos es el de mortalidad, la cual expresa la frecuencia con la cual ocurren las defunciones en una
población dada, en el caso de Guatemala, para el año 2019 fue de 4.72% (Datosmacro, 2021), es decir en promedio 5 muertes por cada 1000 habitantes.
A continuación se realizará un análisis exploratorio, sobre los datos de defunciones reportados por el Instituto Nacional de
Estadística de Guatemala -INE- de los años 2011 a 2020, y dentro de los cuales se pretende encontrar una situación problemática
la cual a su vez pueda ser resuelta con los datos analizados.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sp
from scipy.stats import contingency
import seaborn as sns
from kmodes.kmodes import KModes
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.metrics import silhouette_samples, silhouette_score
import random
# Estilos
plt.style.use('ggplot')
#sns.set_theme(palette=None)

In [2]:
# General functions
def calculate_frequency(data, column, index='index', head = False, size=20, use = False):
    data_f = pd.DataFrame({
        'frequency': data[column].value_counts(),
        'relative_frequency (%)': data[column].value_counts(normalize=True)*100,
        'relative_acc_frequency': data[column].value_counts(normalize=True).cumsum()
    })
    data_f.reset_index(level=[0], inplace=True)
    data_f.rename(columns={index:column}, inplace=True)
    if head:
        left_aligned_df = data_f.head(size).style.set_properties(**{'text-align': 'center'})
    else:
        left_aligned_df = data_f.style.set_properties(**{'text-align': 'center'})
    display(left_aligned_df)

    if use:
        return data_f

    return None

def is_normal(column, tolerancia=0.05):
    return sp.normaltest(column).pvalue > tolerancia

In [3]:
# Cargar los datos
deaths = pd.read_csv('final.csv')
# Como ya se cuenta con la causa de muerte (categorizada) la descripción sale sobrando
# De igual forma, el periodo de edad se puede obtener mediante la edad así que es info redundante
deaths = deaths.loc[:, deaths.columns != 'Perdif']
deaths = deaths.loc[:, deaths.columns != 'caudef.descrip']

FileNotFoundError: [Errno 2] No such file or directory: 'final.csv'

# Exploración de los datos

## ¿Cuáles son los datos?
Comience describiendo cuantas variables y observaciones tiene disponibles, el tipo de cada una de las variables.

In [ ]:
deaths.shape

Para la elaboración del análisis se cuenta con los datos encontrados en la página del Instituto Nacional de Estadística
de Guatemala -INE- para las defunciones del rango de años desde 2011 a 2020. Al unificar los datos provistos en cada uno
de los años, en total, se cuenta con 809296 observaciones y con un total de 28 variables.

### Clasificación de variables
**Cuantitativas discretas**
- EDADIF → Edad del difunto

**Cualitativas Ordinal**
- ESCODIF → Escolaridad del difunto

**Cualitativa Nominal**
- DEPREG → Departamento de registro
- MUPREG → Municipio de registro
- MESREG → Mes de registro
- AÑOREG → Año de registro
- DEPOCU → Departamento de ocurrencia
- MUPOCU → Municipio de ocurrencia
- AREGAG → Área geográfica de ocurrencia
- SEXO
- DIAOCU → Día de ocurrencia
- MESOCU → Mes de ocurrencia
- AÑOOCU → Año de ocurrencia
- PUEDIF → Pueblo de pertenencia del difunto
- ECIDIF → Estado civil del difunto
- CIUODIF → Ocupación del difunto
- PNADIF → País de nacimiento
- DNADIF → Departamento de nacimiento
- MNADIF → Municipio de nacimiento
- NACDIF → Nacionalidad del difunto
- PREDIF → País de residencia
- DREDIF → Departamento de residencia
- MREDIF → Municipio de residencia
- CAUDEF → Causa de defunción
- ASIST → Asistencia recibida
- OCUR → Sitio de ocurrencia
- CERDEF → Quién certifica

## Preprocesamiento 
Análisis de valores atípicos y tratamientos de valores faltantes

Cabe destacar que, aunque los datos sobre los defunciones fueron recolectados de la misma fuente, los datos seguían diferentes esquemas.
Por ejemplo, a partir del año 2012 se dejó de documentar la "ubicación geográfica". Otro desafío que se encontró fue la ocupación.
Aunque a lo largo de todos los años se agrupaban las diferentes ocupaciones de los difuntos en diferentes grupos, las clasificaciones cambiaron entre los años.
En los años 2011-2013 habían 188 categorías de empleo, pero a partir del año 2014 únicamente se trabajó con 45 categorías de ocupación.
En estos casos, simplemente se convirtió manualmente las antiguas ocupaciones a su equivalente más cercano de las nuevas ocupaciones.

Fuera de esto, también se unificaron los nombres de los diferentes departamentos. Por ejemplo, en algunos años lo documentaron como
"Quiche" y en otros "Quiché". Finalmente, para facilitar la realización de cálculos matemáticos, se reemplazó el valor "Ignorado"
de la edad del difunto por -1.

In [ ]:
# Debido a que edad se tomará como variable cuantitativa será necesario reeplazar el valor de ignorado por -1 para que todos
# los datos estén en el mismo formato
deaths['Edadif'] = deaths['Edadif'].replace(['Ignorado'], -1)
deaths['Edadif'] = deaths['Edadif'].astype('int')

In [ ]:
#Se observó que los datos que registraban los departamentos en algunos años se escribían con ascento y en otros no


In [ ]:
deaths['cause'] = deaths['Caudef'].copy()

# deaths.cause = deaths.cause.str.replace(r'\d+.*', r'\1', regex=True)
repl = {r'([AB])(\d+.*)' : r'AB',
        r'(C)(\d+.*)' : r'CD',
        r'(D[0-4])(\d+.*)' : r'CD',
        r'(D[5-8])(\d+.*)' : r'DD',
        r'(E)(\d+.*)' : r'EE',
        r'(F)(\d+.*)' : r'FF',
        r'(G)(\d+.*)' : r'GG',
        r'(H[0-5])(\d+.*)' : r'HA',
        r'(H[6-9])(\d+.*)' : r'HB',
        r'(I)(\d+.*)' : r'II',
        r'(J)(\d+.*)' : r'JJ',
        r'(K)(\d+.*)' : r'KK',
        r'(L)(\d+.*)' : r'LL',
        r'(M)(\d+.*)' : r'MM',
        r'(N)(\d+.*)' : r'NN',
        r'(O)(\d+.*)' : r'OO',
        r'(P)(\d+.*)' : r'PP',
        r'(Q)(\d+.*)' : r'QQ',
        r'(R[0-8])(\d+.*)' : r'RR',
        r'(R9)(\d+.*)' : r'RA',
        r'([ST])(\d+.*)' : r'ST',
        r'([VWXY])(\d+.*)' : r'VY',
        r'([Z])(\d+.*)' : r'ZZ',
        r'([U])(\d+.*)' : r'UU',
        }
deaths.cause = deaths['cause'].replace(repl, regex=True)
deaths['cause']

Para utilizar las causas de las muertes en los análisis de los datos, debido a que existían varias categorías,
se decidió agregar una capa extra de abstracción de las causas, esto principalmente para analizar las causas de muertes
como naturales o por accidentes y sobre enfermedades generales. La nueva categorización de las causas de muerte se guardó
en una nueva columna con el nombre de *cause* y sus valores identifican las siguientes causas:
- AB → Enfermedades infeccionas y parasitarias
- CD → Neoplasias (Cáncer)
- DD → Enfermedades de la sangre y órganos hematopoyéticos y ciertos trastornos que afectan al mecanismo inmunológico
- EE → Enfermedades endocrinas, nutricionales y metabólicas
- GG → Enfermedades del sistema nervioso
- H0 → Enfermedades del ojo y sus anexos
- H6 → Enfermedades del oído y de la apósfisis mastoides
- II → Enfermedades del aparato circulatorio
- JJ → Enfermedades del aparato respiratorio
- KK → Enfermedades del aparato digestivo
- LL → Enfermedades de la piel y del tejido subcutáneo
- MM → Enfermedades del aparato musculoesquelético y del tejido conectivo
- NN → Enfermedades del aparato genitourinario
- OO → Embarazo, parto, puerperio
- PP → Ciertas afecciones originadas en el periodo perinatal
- QQ → Malformaciones congénitas, deformidades y anomalías cromosómicas
- RR → Sintomas, signos y resultados anormales de pruebas complementarias, no clasificados bajo otro concepto
- ST → Lesiones traumáticas, envenenamiento y otras consecuencias de causas externas
- VY → Causas externas de morbilidad
- ZZ → Factores que influyen en el estado de salud y contacto con los servicios sanitarios
- UU → Propósitos especiales

## Variables cuantitativas y su distribución
Haga un resumen de las variables numéricas e investigue si siguen una distribución normal.


In [ ]:
display(deaths['Edadif'].describe())

if is_normal(deaths['Edadif']):
    print(f'La edad tiene una distribución normal.')
else:
     print(f'La edad NO tiene una distribución normal.')

Como se puede notar en los resultados anteriores, la edad promedio para las muertes reportadas de 2011 a 2020 es de
aproximadamente 54 años, eso quiere decir que de los datos reportados, la mayoría están en un rango de fallecer a los
53 a 54 años, esto también se puede notar en que el 50% de los difuntos están por debajo de 60 años de estos un 25% se
encuentra por debajo de 31.

## Variables cualitativas y su frecuencia
Elabore tablas de frecuencia para las variables categóricas, escriba lo que vaya encontrando.

### Departamento de registro
Guatemala presenta la mayor cantidad de muertes registradas. Esto era de esperarse, puesto que la densidad poblacional es
mucho mayor en comparación con los demás departamentos. Luego le sigue Quetzaltenango, al igual que Guatemala, este es
un departamento con una alta densidad poblacional.

In [ ]:
depreg = calculate_frequency(deaths, 'Depreg', use=True)

plt.figure(figsize=(10, 6))
ax = sns.barplot(x='Depreg', y='frequency', data=depreg)
ax.set(title='Gráfico 1. Registro de defunciones por departamento de 2011-2020',
       xlabel= 'Departamento de registro',
       ylabel='Frecuencia')
plt.xticks(rotation=90)
plt.show()

del depreg, ax

### Municipio de registro

In [ ]:
# Para no mostrar muchos datos, solo poner un head de 20
calculate_frequency(deaths, 'Mupreg', head=True)

### Mes de registro

In [ ]:
calculate_frequency(deaths, 'Mesreg')

### Año de registro
La mayor cantidad de muertes se dió en el 2020. Podríamos relacionarlo al inicio de la Pandemia, puesto que hubo un aumento
a comparación del 2019 de 9,624 muertes

In [ ]:
calculate_frequency(deaths, 'Añoreg')

### Departamento de ocurrencia

In [ ]:
calculate_frequency(deaths, 'Depocu')

### Municipio de ocurrencia

In [ ]:
calculate_frequency(deaths, 'Mupocu', head=True)

### Área geográfica de ocurrencia

In [ ]:
calculate_frequency(deaths, 'Areag')

### Sexo

In [ ]:
calculate_frequency(deaths, 'Sexo')

### Día de ocurrencia

In [ ]:
calculate_frequency(deaths, 'Diaocu')

### Mes de ocurrencia

Como se puede observar en la **gráfica 7**, los meses 3 con mayores defunciones son:
- Julio con 73553 muertes reportas,
- Agosto con 69865
- Diciembre con 69332

Es curioso ver como en el mes de julio es donde se dan la mayor cantidad de muertes. Podemos relacionarlo con el pago de
la bonificación del bono 14, ya que en este mes las extorsiones aumentan. Esto nos indica que el índice de violencia puede
ser superior por dicho bono, por lo que las muertes aumentan. Para julio de 2020 además, se tuvieron picos en la pandemia
Covid-19, lo que quizá pudo darle peso a esta variable.

In [ ]:
mesocu = calculate_frequency(deaths, 'Mesocu', use=True)

ax = sns.barplot(x='Mesocu', y='frequency', data=mesocu);
ax.set(title='Gráfico 2. Defunciones de 2011-2020 por mes de ocurrencia',
       xlabel= 'Mes de ocurrencia',
       ylabel='Frecuencia')
plt.xticks(rotation=90)
plt.show()

del mesocu, ax

### Año de ocurrencia

In [ ]:
calculate_frequency(deaths, 'Añoocu')

### Pueblo de pertenencia

In [ ]:
calculate_frequency(deaths, 'Puedif')

### Estado civil del difunto

#### ¿Cuál es el estado civil predominante en las muertes?
Como se puede notar en el gráfico inferior, el estado civil predominante en las defunciones es el soltero. Este es un resultado esperado, ya que
dentro de los datos se encuentran personas de todas las edades, por lo que muchos de los fallecidos son aún niños cuando mueren, por lo tal, su
estado civil es soltero, y también están todos aquellos que mueren a una edad relativamente joven por lo que aún no han contraído matrimonio.

In [ ]:
estado = calculate_frequency(deaths, 'Ecidif', use=True)

ax = sns.barplot(x='Ecidif',y='frequency',data=estado)
ax.set(title='Gráfico 3. Estado civil de los difuntos reportados de 2011-2020',
       xlabel= 'Estado civil',
       ylabel='Frecuencia')
plt.xticks(rotation=90)
plt.show()

del estado, ax

### Escolaridad del difunto
¿Influye el tener una educación superior a reducir el número de defunciones?
La mayoría de muertes se dan en personas con escolaridad nula. Es interesante observar como estudiantes
universitarios y de posgrado son los que presentan la menor cantidad de muertes en los últimos 10 años.

In [ ]:
escodif = calculate_frequency(deaths, 'Escodif', use=True)

ax = sns.barplot(x= 'Escodif', y='frequency', data=escodif)
ax.set(title='Gráfico 4. Escolaridad de los difuntos reportados de 2011-2020',
       xlabel= 'Escolaridad',
       ylabel='Frecuencia')
plt.xticks(rotation=90)
plt.show()

del escodif, ax

### Ocupación del difunto

In [ ]:
calculate_frequency(deaths, 'Ciuodif', head=True)

### País de nacimiento

In [ ]:
calculate_frequency(deaths, 'Pnadif', head=True)

### Departamento de nacimiento

In [ ]:
calculate_frequency(deaths, 'Dnadif')

### Municipio de nacimiento

In [ ]:
calculate_frequency(deaths, 'Mnadif', head=True)

### Nacionalidad del difunto

In [ ]:
calculate_frequency(deaths, 'Nacdif')

### País de residencia

In [ ]:
calculate_frequency(deaths, 'Predif')

### Departamento de residencia

In [ ]:
calculate_frequency(deaths, 'Dredif')

### Municipio de residencia

In [ ]:
calculate_frequency(deaths, 'Mredif', head=True)

### Causa de defunción

In [ ]:
calculate_frequency(deaths, 'cause', head=True, size=30)

### Asistencia recibida

#### ¿Cuál es el tipo de asistencia recibida predominante en las defunciones?
Como se puede observar en el gráfico 2, en un 48% de las defunciones no se recibió ningún tipo de asistencia, sin
embargo, algo interesante a notar es que un 47% de los difuntos recibieron asistencia médica. Dejando así solo aproximadamente
un 1% de diferencia entre la asistencia predominante y la segunda más utilizada.


In [ ]:
assist = calculate_frequency(deaths, 'Asist', use=True)

ax = sns.barplot(x='Asist',y='frequency',data=assist)
ax.set(title='Gráfico 5. Tipo de asistencia recibida por los difuntos reportados de 2011-2020',
       xlabel= 'Tipo de asistencia',
       ylabel='Frecuencia')
plt.xticks(rotation=90)
plt.show()
del assist, ax

### Sitio de ocurrencia

### ¿Cuál es la proporción de personas que mueren en sus casas con respecto a las que mueren en hospitales?
Como se puede observar en el **gráfico 3** el principal sitio de ocurrencia de las muertes en en los domicilios de los difuntos, estos
representando un 69% de las muertes reportadas, y es destacable notar que únicamente un 21% de los difuntos fallecieron en el hospital.


In [ ]:
ocur = calculate_frequency(deaths, 'Ocur', use=True)

fig, ax = plt.subplots(figsize=(7, 7), subplot_kw=dict(aspect="equal"))
explode = (0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.05, 0.05)
wedges, texts = ax.pie(ocur['relative_frequency (%)'], wedgeprops=dict(width=0.6), startangle=-30, explode=explode)

bbox_props = dict(boxstyle="square,pad=0.5", fc="w", ec="k", lw=0.77)
kw = dict(arrowprops=dict(arrowstyle="-"),
          bbox=bbox_props, zorder=0, va="center")

for i, p in enumerate(wedges):
    ang = (p.theta2 - p.theta1)/5. + p.theta1
    y = np.sin(np.deg2rad(ang))
    x = np.cos(np.deg2rad(ang))
    horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
    connectionstyle = "angle,angleA=0,angleB={}".format(ang)
    kw["arrowprops"].update({"connectionstyle": connectionstyle})
    ax.annotate(ocur['Ocur'][i] + ' ' + str(round(ocur['relative_frequency (%)'][i], 3)) + '%', xy=(x, y), xytext=(1.5*np.sign(x), 1.5*y),
                horizontalalignment=horizontalalignment, **kw)
plt.title('Gráfico 6. Porcentaje de sitio de muerte para los difuntos reportados entre 2011-2020')
plt.show()

del ocur, fig, ax, explode, wedges, texts, bbox_props, kw, 

### Quién certifica

In [ ]:
calculate_frequency(deaths, 'Cerdef')

## Definiendo elementos clave
Cruce las variables que considere que son las más importantes para hallar los elementos clave que lo pueden llevar a
comprender lo que está causando el problema encontrado.

Para hacer el cruce de las variables y comprender si existe cierta asociación entre dichas variables se utilizará Cramer´s V
la cual es una medida de la longitud entre variables categóricas, básicamente mide la relación entre las variables. Su valor
varía de 0 a 1, donde 0 indica que no existe ningún tipo de asociación entre las variables y 1 que sí existe asociación entre
las mismas. Por otro lado, también se utilizarán las tablas de contingencia para hacer el "cruce" de las variables.

Durante el análisis exploratorio de los datos y mediante las diversas preguntas que se plantearon para conocer dichos datos
se pudo encontrar que existe una gran parte de las defunciones las cuales NO reciben asistencia médica, y esto es más notorio
cuando se habla de asistencia médica por pueblo de pertenencia. Por lo tal, se evaluará únicamente la asociación de las variables
con respecto al tipo de asistencia que reciben.

In [ ]:
depreg = pd.crosstab(deaths['Depreg'], deaths['Asist'])
print(depreg)
print('Asociación: ', contingency.association(depreg))

del depreg


In [ ]:
mupreg = pd.crosstab(deaths['Mupreg'], deaths['Asist'])
print(mupreg)
print('Asociación: ', contingency.association(mupreg))

del mupreg

In [ ]:
mesreg = pd.crosstab(deaths['Mesreg'], deaths['Asist'])
print(mesreg)
print('Asociación: ', contingency.association(mesreg))

del mesreg

In [ ]:
areag = pd.crosstab(deaths['Areag'], deaths['Asist'])
print(areag)
print('Asociación: ', contingency.association(areag))

del areag


In [ ]:
sexo = pd.crosstab(deaths['Sexo'], deaths['Asist'])
print(sexo)
print('Asociación: ', contingency.association(sexo))

del sexo


In [ ]:
puedif = pd.crosstab(deaths['Puedif'], deaths['Asist'])
print(puedif)
print('Asociación: ', contingency.association(puedif))

del puedif


In [ ]:
ecidif = pd.crosstab(deaths['Ecidif'], deaths['Asist'])
print(ecidif)
print('Asociación: ', contingency.association(ecidif))

del ecidif


In [ ]:
escodif = pd.crosstab(deaths['Escodif'], deaths['Asist'])
print(escodif)
print('Asociación: ', contingency.association(escodif))

del escodif


In [ ]:
ciuodif = pd.crosstab(deaths['Ciuodif'], deaths['Asist'])
print(ciuodif)
print('Asociación: ', contingency.association(ciuodif))

del ciuodif


In [ ]:
nacdif = pd.crosstab(deaths['Nacdif'], deaths['Asist'])
print(nacdif)
print('Asociación: ', contingency.association(nacdif))

del nacdif


In [ ]:
caudef = pd.crosstab(deaths['Caudef'], deaths['Asist'])
print(caudef)
print('Asociación: ', contingency.association(caudef))

del caudef


In [ ]:
ocur = pd.crosstab(deaths['Ocur'], deaths['Asist'])
print(ocur)
print('Asociación: ', contingency.association(ocur))

del ocur


In [ ]:
cerdef = pd.crosstab(deaths['Cerdef'], deaths['Asist'])
print(cerdef)
print('Asociación: ', contingency.association(cerdef))

del cerdef

Con base en la problemática actualmente planteada y la asociación entre las variables encontrada anteriormente se decide
que para el análisis de clustering se utilizaran las siguientes variables:
- Asistencia
- Departamento de registro
- Municipio de registro
- Población de pertenencia
- Área geográfica
- Causa de defunción
- Sitio de Ocurrencia
- Quien certifica

## Estado de los datos
Haga gráficos exploratorios que le dé ideas del estado de los datos.

#### ¿Cuál es la relación entre el género y la edad en cuanto a las defunciones en Guatemala? ¿Las mujeres tienden a morir más jóvenes que los hombres?

En el gráfico 4, con relación al género y edad, se puede observar que hasta los 80 años, se han reportado más muertes masculinas que femeninas en cada uno de los rangos de edad.
Sin embargo, luego de los 80 años la relación cambia, y en este caso, se reportaron más muertes de mujeres que de hombres. Por lo tal, no se puede inferir que las mujeres tienden
a morir más jóvenes que los hombres ya que por lo contrario, se han reportado más muertes de hombres jóvenes que de mujeres.

In [ ]:
# Debido a que es una diversa cantidad de edades, se decide hacer grupos
new_deaths = deaths.copy()

for x in range(11):
    if x < 10:
        new_deaths.loc[(new_deaths.Edadif >= x * 10) & (new_deaths.Edadif < (x+1) * 10), 'Edadif'] = x
    else:
       new_deaths.loc[(new_deaths.Edadif >= 100), 'Edadif'] = 10

age_gender = new_deaths.groupby(by=['Sexo', 'Edadif']).count()
age_gender.reset_index(level=[1], inplace=True)

ax = sns.barplot(x="Edadif", y="Depreg", hue=age_gender.index, data=age_gender)
ax.set(title='Gráfico 7. Relación de edad-género para defunciones reportadas entre 2011-2020',
       xlabel= 'Rango de edad',
       ylabel='Frecuencia')
plt.xticks(rotation=90)
plt.show()

del new_deaths, age_gender, ax

#### ¿Cuál es el porcentaje de población indígena que recibe asistencia hospitalaria?

Como se muestra en el **gráfico 5** solamente un 24% de las defunciones reportadas del pueblo indígena han recibido asistencia médica
a diferencia de un 70% de los cuales no ha recibido ningún tipo de asistencia.

In [ ]:
etnic_asist = deaths.groupby(by=['Puedif', 'Asist']).count()
etnic_asist.reset_index(level=[1], inplace=True)
etnic_asist = etnic_asist.loc[ :, ['Asist', 'Depreg']]
etnic_asist['frequency (%)'] = (etnic_asist['Depreg'] / etnic_asist['Depreg'].sum()) * 100

etnic_asist_ind = etnic_asist.loc[etnic_asist.index == 'Indigena']
plt.pie(etnic_asist_ind.Depreg, labels = etnic_asist_ind.Asist, autopct='%.0f%%')
plt.title('Gráfico 8. Asistencia recibida por población indígena en las defunciones de 2011-2020')
plt.show()

etnic_asist_noind = etnic_asist.loc[etnic_asist.index == 'No indigena']
plt.pie(etnic_asist_noind.Depreg, labels = etnic_asist_noind.Asist, autopct='%.0f%%')
plt.title('Gráfico 9. Asistencia recibida por población no indígena en las defunciones de 2011-2020')
plt.show()

del etnic_asist, etnic_asist_noind, etnic_asist_ind

#### ¿Cómo es la distribución de las muertes de los jóvenes entre 18-30 que tienen una escolaridad por encima de diversificado?

De forma sencilla de interpretar el gráfico 6 representa la distribución de las muertes de personas entre 18-30 años agrupados por escolaridad. Como se puede notar,
con personas con grado académico igual y por encima de diversificado, la mayor cantidad de muertes se da justamente con grado académico de diversificado. Este resultado
es bastante interesante ya que si bien esto no presenta una relación entre nivel académico y muertes sino más bien, representa la desigualdad en acceso a educación que se vive
dentro del país, ya que como se puede notar, la cantidad de jóvenes, que fallecen y son universitarios es muy poca en comparación con aquellos que mueren y están estudiando o tienen
grado académico de primaria.

In [ ]:
young = deaths.loc[(deaths['Edadif'] >= 18) & (deaths['Edadif'] <= 30), ]
young = young.groupby(by=['Escodif']).count()

ax = sns.barplot(x=young.index, y="Depreg", data=young)
ax.set(title='Gráfico 10. Escolaridad de las muertes jóvenes (18-30años) reportadas entre 2011-2020',
       xlabel= 'Escolaridad',
       ylabel='Frecuencia')
plt.xticks(rotation=90)
plt.show()

del young, ax

### ¿En qué meses se dieron la mayor cantidad de defunciones?

Como se puede observar en la **gráfica 7**, los meses 3 con mayores defunciones son:
- Julio con 73553 muertes reportas,
- Agosto con 69865
- Diciembre con 69332

In [ ]:
month = deaths.groupby(["Mesocu"])["Mesocu"].count().sort_values(ascending=False)
display(month)

month.plot.bar()
plt.title('Gráfico 11. Defunciones reportadas entre 2011-2020 separadas por mes')
plt.xlabel('Mes')
plt.ylabel('Frecuencia')

del month

### ¿En qué países ocurrieron la mayor cantidad de defunciones en los años 2019 y 2020 (pandemia)?
El país con mayor defunciones es Guatemala. Cabe mencionar que este resultado se debe, principalmente, a que la mayoría
de datos que se recolectaron fueron dentro del país, obviando así muchos casos de defunciones de guatemaltecos que residen
en el extranjero.

In [ ]:
pandemia = pd.DataFrame(deaths.query("Añoreg>=2019"))
pandemia['Edadif'] = [-1 if year == "Ignorado" else year for year in pandemia["Edadif"]]
pais = pandemia.groupby(["Predif"])["Predif"].count().sort_values(ascending=False).head()
display(pais)

pais.plot.bar()
plt.title('Gráfico 12. Residencia del país del difunto de las muertes reportadas entre 2011-2020')

del pais

### Durante estos años, las personas que fallecieron tuvieron tratamiento médico? 

Como se puede observar, la cantidad de defunciones que hubieron en estos años de pandemia y que recibieron asistencia médica
fueron de 93212 muertes, luego siguen las personas que no recibieron ningún tipo de asistencia médica. Esto se puede deber
a que en esos tiempo, muchas personas se enfermaban y cuando necesitaban ir a un hospital, estos se encontraban llenos y
no lograban encontrar cupo.

In [ ]:
display(pandemia.groupby("Asist")["Asist"].count().sort_values(ascending=False))

del pandemia

### ¿Existe alguna relación entre el sexo y la atención recibida que tuvieron los fallecidos en los años 2011 a 2020?

In [ ]:
atencionGenero = deaths.groupby(["Asist","Sexo"])["Asist"].count().sort_values(ascending=False)
display(atencionGenero)

atencionGenero.plot.bar()
plt.title('Gráfico 13. Atención recibida por sexo en las defunciones reportadas de 2011-2020')
plt.xlabel('Asistencia, Sexo')
plt.ylabel('Frecuencia')

del atencionGenero

Como se puede observar en la **gráfica 9**, no hay alguna relación entre el género y la atención recibida, por lo que no
se puede concluir que el género afecte en algo hacia la atención médica que se puede recibir. Sin embargo, se puede notar
que sin importar el género, existe una gran cantidad de personas que no recibe atención médica antes de morir.

### ¿En Guatemala, en qué lugares predominan las defunciones? ¿El servicio que ofrecen los hospitales privados es mejor que el de los hospitales públicos? 

In [ ]:
guatemala = pd.DataFrame(deaths.query("Predif=='Guatemala'"))
municipios = guatemala.groupby("Mredif")["Mredif"].count().sort_values(ascending=False).head(10)
display(municipios)

municipios.plot.pie(autopct='%.0f%%')
plt.title('Gráfica 14. Predominancia de las defunciones reportadas para 2011-2020 por municipio de residencia')
plt.ylabel('')
plt.xlabel('')

In [ ]:
lugar = guatemala.groupby("Ocur")["Ocur"].count().sort_values(ascending=False).head(10)
display(lugar)

lugar.plot.pie()
plt.title('Gráfico 11. Lugar de ocurrencia reportado para las defunciones entre 2011-2020')
plt.ylabel('')
plt.xlabel('')

del guatemala, lugar, municipios

Cómo se puede observar en las dos gráficas de pie, el municipio con mayor cantidad de defunciones es Guatemala, luego le
sigue Mixco, Villa Nueva y Quetzaltenango. Por otro lado, se puede ver en el **gráfico 11** en los hospitales públicos
existió una mayor cantidad de defunciones que en hospitales privados, lo que podría indicar que el servicio de los
hospitales privados es mejor, sin embargo, como se vio en la pregunta anterior y en la siguiente pregunta,
el número de personas que tiene acceso a tratamientos médicos privados es muy baja,
por lo tal no es posible concluir que los hospitales privados son mejores que los públicos debido a que la cantidad
de personas que cuentan con los recursos económicos para asistir a un hospital privado es menos de la mitad de la población
guatemalteca.

### ¿Las personas fallecidas de 50 años o más recibieron buen tratamiento médico? 

In [ ]:
datos2 = deaths.copy()
datos2['Edadif'] = [-1 if year == "Ignorado" else int (year) for year in datos2["Edadif"]]
mayores = pd.DataFrame(datos2.query("Edadif>49"))

display(mayores.groupby("Asist")["Asist"].count().sort_values(ascending=False))

del mayores, datos2

Cómo se mencionó anteriormente, aquí se puede observar que las personas mayores a los 50 años no recibieron asistencia
médica a la hora de su muerte, con una cantidad de 200701 defunciones registradas para ese caso. Luego siguen las
defunciones que sí recibieron asistencia médica, con un valor de 196730.

### ¿Quienes fueron las personas que más certificaron durante los años 2015 a 2020?

In [ ]:
datos2 = pd.DataFrame(deaths.query("Añoreg>2014"))

cert = datos2.groupby("Cerdef")["Cerdef"].count().sort_values(ascending=False)
display(cert)

cert.plot.bar()
plt.title('Gráfico 15. Certificadores reportados para las defunciones entre 2015 a 2020')
plt.xlabel('Certificador')

del datos2

Como se puede observar en la gráfica de arriba, las personas que mayor certificaron son los médicos, seguidos por los
que se registraron como "ignorados".

### ¿Cuál es el porcentaje de las muertes por enfermedad que NO recibieron asistencia médica?
En el **gráfico 16** se puede observar que aproximadamente un 46% de las muertes a causa natural reportadas durante los años
2011 a 2020 no recibieron asistencia médica.

In [ ]:
illness = pd.DataFrame({
    'Asist': deaths['Asist'],
    'cause' : deaths['cause'].replace(r'([A-R])([A-R])', r'NC', regex=True),
    'count': 1
}).groupby(['cause', 'Asist']).count()
illness.reset_index(level=[1], inplace=True)
illness = illness.loc[illness.index == 'NC']
illness['frequency (%)'] = (illness['count'] / illness['count'].sum()) * 100
display(illness)

plt.pie(illness['frequency (%)'], labels = illness.Asist, autopct='%.0f%%')
plt.title('Gráfico 16. Asistencia recibida por muertes naturales de los difuntos ')
plt.show()

del illness

### ¿Cuál es el porcentaje de las muertes por causa externa (i.e. no causa natural) que recibieron asistencia médica?
Como se puede notar en el **gráfico 17** únicamente un 40% de las muertes reportadas por causas externas recibieron asistencia
médica, dejando así, un 55% de difuntos sin asistencia médica.

In [ ]:
illness = pd.DataFrame({
    'Asist': deaths['Asist'],
    'cause' : deaths['cause'].replace(r'([S-Y])([S-Y])', r'EC', regex=True),
    'count': 1
}).groupby(['cause', 'Asist']).count()
illness.reset_index(level=[1], inplace=True)
illness = illness.loc[illness.index == 'EC']
illness['frequency (%)'] = (illness['count'] / illness['count'].sum()) * 100
display(illness)

plt.pie(illness['frequency (%)'], labels = illness.Asist, autopct='%.0f%%')
plt.title('Gráfico 17. Asistencia recibida por muertes naturales de los difuntos ')
plt.show()

del illness


## Agrupamiento
Haga un agrupamiento (clustering) e interprete los resultados.
Para la elaboración del agrupamiento, debido a que se cuenta con una gran cantidad de datos, sin embargo no se tiene a
disposición un ordenador capaz de procesar dicha cantidad de datos, entonces se decidió utilizar una muestra de la población.

Debido a que se busca que la muestra sea lo más significativa posible, se decidió tomar aproximadamente un 15% de los datos
de cada año y así garantizar que en la muestra existirá una proporción de defunciones de todos los años descritos en la población
original.

Al ser únicamente variables categóricas se utilizará como algoritmo de agrupamiento k-modes. El cual es una variante de k-means
pero en lugar de tomar la distancia entre los datos, este se basa en la moda de los mismos.

In [ ]:
# Obtención de la muestra
deaths_sample = deaths.groupby('Añoocu', group_keys=False).apply(lambda x: x.sample(frac=0.15, random_state=191943))

usable = ['Asist', 'Depreg', 'Mupreg', 'Añoreg', 'Puedif', 'cause', 'Ocur', 'Cerdef']
deaths_sample = deaths_sample[usable]

### Número óptimo de clusters.

In [ ]:
# Elbow curve to find optimal K
cost = []
K = range(1,8)
for num_clusters in list(K):
    kmode = KModes(n_clusters=num_clusters, init="random", n_init = 5, verbose=1)
    kmode.fit_predict(deaths_sample)
    cost.append(kmode.cost_)
    
plt.plot(K, cost, 'bx-')
plt.xlabel('Número de Clusters')
plt.ylabel('Valor')
plt.title('Método de Codo para el Número óptimo de Clusters K-Modes')

Mediante el gráfico del codo se puede observar que para el conjunto de datos a agrupar el número óptimo de clusters
es 3, ya que es en este punto que al aumentar la cantidad de clusters el valor no cambia significativamente.

### Uso de algoritmo de agrupamiento.

In [ ]:
kmode = KModes(n_clusters=3, init="random", n_init = 5, verbose=1, random_state=42)
clusters = kmode.fit_predict(deaths_sample)

In [ ]:
binary_sample = pd.get_dummies(deaths_sample)

### Calidad del agrupamiento usando el método de la silueta.

In [ ]:
# plt.figure.size_inches(18, 7)
plt.xlim([-0.1, 1])
# The (n_clusters+1)*10 is for inserting blank space between silhouette
# plots of individual clusters, to demarcate them clearly.
# plt.set_ylim([0, len(X) + (n_clusters + 1) * 10])

silhouette_avg = silhouette_score(binary_sample, clusters)
print("For n_clusters = 3 The average silhouette_score is :", silhouette_avg)

# Compute the silhouette scores for each sample
sample_silhouette_values = silhouette_samples(binary_sample, clusters)

y_lower = 10
for i in range(3):
    ith_cluster_silhouette_values = sample_silhouette_values[clusters == i]
    ith_cluster_silhouette_values.sort()
    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i

    # color = cm.nipy_spectral(float(i) / n_clusters)
    plt.fill_betweenx(
        np.arange(y_lower, y_upper),
        0,
        ith_cluster_silhouette_values,
        facecolor='red',
        edgecolor='black',
        alpha=0.7,
    )

    # Label the silhouette plots with their cluster numbers at the middle
    plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

    # Compute the new y_lower for next plot
    y_lower = y_upper + 10  # 10 for the 0 samples

plt.title("The silhouette plot for the various clusters.")
plt.xlabel("The silhouette coefficient values")
plt.ylabel("Cluster label")

# The vertical line for average silhouette score of all the values
plt.axvline(x=silhouette_avg, color="red", linestyle="--")

plt.yticks([])  # Clear the yaxis labels / ticks
plt.xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

plt.show()

In [ ]:
# Agregando los resultados del cluster al dataframe
deaths_sample['cluster'] = clusters

### Interpretación de los grupos, usando para eso las variables numéricas y categóricas dentro de cada grupo.
Para hacer el análisis y la interpretación de los resultados del cluster se utilizó la visualización de los mismos, es
decir, se observó gráficamente cómo se había realizado la división en los grupos mediante gráfico de barras.

In [ ]:
variables = ['Asist', 'Depreg', 'Añoreg', 'Puedif', 'cause', 'Ocur', 'Cerdef']

for variable in variables:
    plt.figure(figsize=(12,6))
    ax = sns.countplot(x="cluster", hue=variable, data=deaths_sample)
    plt.legend(bbox_to_anchor=(1.05, 1),
               loc='upper left',
               borderaxespad=0,
               title=variable,
               fontsize='medium',
               title_fontsize='large')
    plt.show(ax)

En las gráficas anteriores se puede notar que si bien en cada grupo no se encuentra una sola categoría de cada variable
sin embargo, si es fácil de observar que existe una categoría que predomina.

En el caso del grupo 1, predominan las personas que no recibieron ningún tipo de asistencia a la hora de morir, especialmente en los
departamentos de Huehuetenango, San Marcos y Alta Verapaz, siendo su principal población perteneciente al pueblo indígena.
Como principales causas de muerte las enfermedades del aparato circulatorio y enfermedades del aparato respiratorio. Estas muertes
en mayoría ocurrieron en los domiciolios de los difuntos. Y debido a esto, se desconoce quién hizo la certificación de muerte.

Por otro lado se tiene al grupo 2, en el cual se encuentran los difuntos que recibieron asistencia médica, principalmente
en el departamento de Guatemala alrededor del año 2019 - 2020 siendo en su mayoría personas no indígenas con causa principal
de muerte por enfermedades del aparato circulatorio seguido por enfermedades metabólicas, endocrinas y nutricionales.
Estas muertes principalmente han ocurrido en hospitales públicos y domicilios y por lo tal, han sido en su mayoría, certificadas por un
médico.

Finalmente se tiene al grupo 3, este cuenta con observaciones que en su mayoría  no recibieron asistencia médica, siendo estos principalmetne
residentes del departamento de Guatemala alrededor del año 2015, en su mayoría es población no indígena cuya muerte ha ocurrido principalmente a causa
de situaciones externas tales como accidentes automovilísticos, lesiones autoinflingidas o lesiones a causua de accidentes.
En este grupo predominan los fallecimientos en los domicilios y la certificación ha sido dada, en su mayoría, por un médico.

# Análisis de resultados

## Situación problemática
Describa la situación problemática que lo lleva a acotar un problema a resolver.

Durante el análisis exploratorio realizado en el apartado anterior se pudo encontrar que aproximadamente el 50% de los datos
reportados para las defunciones de los años de 2011 a 2020 no recibe ningún tipo de asistencia al morir. De igual forma se buscó
analizar si existía algún tipo de sesgo en cuanto a la asistencia dependiendo del pueblo de pertenencia del difunto, y como se
puede observar en el **gráfico 5.1** y **gráfico 5.2** en el caso de la población indígena, solamente un 24% de la población
recibe asistencia médica, a diferencia de la población no indígena, de la cual aproximadamente un 57% recibe asistencia médica.
Por otro lado, algo interesante a notar es que el sexo no establece un marco de sesgo para la atención esto es significativo
debido al nivel de desiugaldad de género que se vive en el país.

## Problema científico
Enuncie un problema científico y unos objetivos preliminares

El acceso a salud de calidad se encuentra estipulado dentro de la Constitución de Guatemala en el cual se asegura
que todos los seres humanos desde el momento de nacer tienen el derecho de recibir asistencia médica de manera gratuita,
En el boletín No.19 proporcionado por las naciones unidas indican que el derecho a la salud conlleva
a que todas las personas tengan acceso a la salud en el momento que lo necesiten y que además esta sea de calidad.
Sin embargo, en el resumen de *Estrategia de Cooperación* se comenta que Guatemala aún experimenta una crisis institucional
para garantizar el acceso y la cobertura de servicios de salud básicos. Como se mencionó al describir la situación problemática
aproximadamente el 50% de las personas que fallecen no reciben ningún tipo de asistencia médica antes de su muerte, sin embargo,
la problemática radica en conocer si exis

**Objetivos**
- Conocer la diferencia en la asistencia médica recibida por los difuntos dependiendo el departamento de residencia del fallecido
- Descubrir si a causa de la pandemia existieron cambios en la asistencia médica recibida por los guatemaltecos
- Determinar si la causa de fallecimiento contribuye al recibimiento de asistencia médica (i.e. muerte a causa natural vs suicidio)

## ¿Qué se tiene para responder el problema?
Describa los datos que tiene para responder el problema planteado. Esto incluye el estado en que encontró el o los conjuntos
de datos y las operaciones de limpieza que le realizó, en caso de que hayan sido necesarias.

Para resolver el problema se cuenta con la base de datos pública de defunciones del Instituto Nacional de Estadística de Guatemala (INE).
Dicha base de datos nos permite detectar la nacionalidad del difunto, su edad, ocupación, causa de defunción, grupo étnico, entre otros.
Específicamente, se utilizaron los reportes de defunción de los años 2011 a 2020.

Cabe destacar que la base de datos necesitaba de limpieza. Con la limpieza se trabajó con 3 problemas principales:
Diferencias en Clasificación: Como el grupo étnico y la ocupación del difunto.
Nombres de Departamentos: Por ejemplo, Quiché y Quiche
Variables que no están en todos los años: Como el área geográfica y nacionalidad del difunto.
Un ejemplo perfecto de las diferencias entre las clasificaciones es el grupo étnico. Durante los años 2011-2013 se
trabajó con: Indígena y no Indígena, mientras que a partir del año 2014 se utilizó Ladino / Mestizo, Xinca, Garífuna y Maya.
En este caso, simplemente se adaptó todo al esquema antiguo, donde la etnia Maya fue reclasificada a Indígena y todas las demás a No Indígena.
También ocurrió algo similar con las clasificaciones de la ocupación del difunto.

Además, en caso de que no todas las variables estuvieran en todos los años, los valores faltantes fueron rellenados con “nan” o con “Ignorado”.
Cabe destacar que no se utilizaron estas variables durante la realización del estudio, pero todavía se preprocesaron en caso de que sí fuera necesario usarlas.


## Conclusiones
Escriba unas conclusiones con los hallazgos encontrados durante el análisis exploratorio

Una de las cosas más interesantes al analizar los datos, tanto mediante el análisis exploratorio como mediante el agrupamiento
fue el encontrar el "sesgo" o preferencia que existe al proporcionar asistencia médica, ya que esta principalmente se provee a
personas que no pertenecen al pueblo indígena y que residen en el departamento de Guatemala.

Entre los años 2019-2020, la causa de muerte que predominó en el departamento de Guatemala fueron causadas por problemas en el
aparato circulatorio, enfermedades metabólicas, endocrinas y nutricionales. Valdría la pena tomar en cuenta esto para futuras
investigaciones y encontrar cuál es la correlación que existe entre estas causas de muertes y el COVID-19. Ya que como se puede
notar en la agrupación realizada, especialmente el grupo 1, este representa principalmente muertes en el 2019-2020
cuya causa fueron entre otros problemas respiratorios y circulatorios, esto es interesante que durante esos años inició y fue
el auge de la pandemia en Guatemala, y evidentemente aunque existían hospitales especializados para atender las personas contagiadas
en su mayoría no se daban abasto.

Por otro lado, otro aspecto interesante encontrado en la agrupación y en el análisis es que en los últimos años (2019 en adelante)
disminuyó considerablemente las muertes registradas por accidentes o autolesiones, en comparación con años anteriores, y
por el contrario se vio un incremento en las muertes por enfermedades del aparato circulatorio seguido por enfermedades metabólicas, endocrinas y nutricionales